In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Futures Valuation Workflow

Attributes
----------
futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Computing Intraday P&L for Futures with and without Daily Close outs

In this notebook, we demonstrate how P&L can be computed for Futures instruments under two different cash accounting treatments. We look at these treatments over the course of three days for both a Dec 21 and Mar 22 Bund Futures contract. Note that we maintain the same market quotes and transaction amounts for each Futures contract to more easily demonstrate how the PV and P&L measures evolve over time.


### Dec 21 Bund Futures with Unrealized P&L (‘non-close out’)
In our first example, we look at a Futures contract whereby the contract's daily P&L is left as unrealised on the position as opposed to marked-to-market on a daily basis.


### Mar 22 Bund Futures with Realized P&L ('close out’)
In our second example, we look at a Futures contract whereby the previous day's P&L is realized at the beginning of each day and booked down into a separate cash line item. We then reset the cost basis of our holding to reflect this mark-to-market such that the computed PV at the start of each day is 0.

## Table of Contents
* [1. Create Portfolio](#-Create-Portfolio)
* [2. Create Futures](#-Create-Futures)
* [3. Transactions](#-Transactions)
* [4. Quotes](#-Quotes)
* [5. Valuations](#-Valuations)

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam import RefreshingToken
import lusid_sample_data as import_data

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Create API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name="futuresLoader"
)

In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [4]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
transaction_type_scope = import_data.create_scope_id()

# 1. Create Portfolio<a class="anchor" id="-Create-Portfolio"></a>

In [5]:
portfolio_code = "FuturesPortWithDiffCostBasis"

try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'FuturesPortWithDiffCostBasis' because it already exists in scope 'ibor'.


# 2. Create Futures<a class="anchor" id="-Create-Futures"></a>

In [6]:
# Define function that creates futures
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

## 2.1 Create Bund Futures Contract Expiring Dec 21

In [7]:
start_date = datetime(2021, 3, 8, tzinfo=pytz.utc)
effectiveAt = datetime.today().replace(tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "Z"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Dec21"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = None
identifier = "FutBund001"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D6H


## 2.2 Create Bund Futures Contract Expiring Mar 22

In [8]:
start_date = datetime(2021, 6, 8, tzinfo=pytz.utc)
effectiveAt = datetime.today().replace(tzinfo=pytz.utc)
maturity_date = datetime(2022, 3, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "H"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Mar22"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = 0
identifier = "FutBund002"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D6G


# 3. Transactions<a class="anchor" id="-Transactions"></a>

## 3.1 Create Transaction Types

To book Futures transactions under the close out and non-close out method, we need to create three transaction types. 

The first transaction type 'OpenContract' is a standard transaction used to enter into our positions. This contains one movement that increases the size of the position based on a quantity provided.

The second two transaction types 'RealisePnLIncrease' and 'RealisePnLDecrease' are specific to the close out method used by our Mar 22 Futures position. These transactions are there to adjust the cost basis up or down depending on the position's daily P&L.

Each of these transactions have two movements representing the two economic changes to the portfolio:

- The first movement updates the cost basis of the Futures position to reflect the daily mark-to-market change. This is done by adjusting the total_consideration of the position up or down while leaving the quantity of the position unchanged (in the transaction file we specify a quantity of 0). 
- The second movement records the cash generated from the daily mark-to-market change. This again gets its amount from the total_consideration value in the Excel file.

In [9]:
# Before setting the transaction type config, some side definitions are needed in the non-default transaction type scope.
# If working in the default scope, these side definitions are already inserted so this step can be skipped.
default_side_definitions = transaction_configuration_api.list_side_definitions()

def mapSide(side_definition):
    return {"side": side_definition.side, "sideRequest": lusid.models.SideDefinitionRequest(security = side_definition.security, currency = side_definition.currency, rate = side_definition.rate, units = side_definition.units, amount = side_definition.amount, notional_amount= side_definition.notional_amount)}

set_side_definitions_request = [mapSide(side_definition) for side_definition in default_side_definitions.values]
transaction_configuration_api.set_side_definitions(set_side_definitions_request, scope = transaction_type_scope)

# Build and insert the transaction types
transaction_type_requests = [
    models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type="OpenContract",
                description="An FuturesTxnType transaction type",
                transaction_class="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    ),
    models.TransactionTypeRequest(    
        aliases=[        
            models.TransactionTypeAlias(
                type="RealisePnLIncrease",
                description="Increase cost basis and adjust cash",
                transaction_class="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            ),models.TransactionTypeMovement(
                movement_types="CashReceivable",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    ),
    models.TransactionTypeRequest(    
        aliases=[        
            models.TransactionTypeAlias(
                type="RealisePnLDecrease",
                description="Decrease cost basis and adjust cash",
                transaction_class="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties=None,
                mappings=[],
            ),models.TransactionTypeMovement(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    )
]

new_txn_config = transaction_configuration_api.set_transaction_type_source(
    source="default",
    transaction_type_request=transaction_type_requests,
    scope=transaction_type_scope
)

# Call LUSID to update the transaction type scope of your portfolio 
patch_document = [
    {
        "value": transaction_type_scope,
        "path": "/transactiontypescope",
        "op": "add"
    }
]
patch_response = api_factory.build(lusid.api.TransactionPortfoliosApi).patch_portfolio_details(
    scope=scope,
    code=portfolio_code,
    operation=patch_document)

## 3.2 Load Transaction Data

In [10]:
# Read in transaction data read from file
futures_transactions = pd.read_excel("data/futures_data.xlsx", sheet_name="transactions")

In [11]:
for index, row in futures_transactions.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=row['portfolio'],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["trade_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency_id"]
                ),
            )
        ],
    )

# 4. Quotes<a class="anchor" id="-Quotes"></a>

## 4.1 Book Quotes

In [12]:
# Read in futures data from file
futures_prices = pd.read_excel("data/futures_data.xlsx", sheet_name="prices")
# Output futures data
futures_prices

,date,prices,luid,id_type,currency,scaling_factor,note
0,2021-09-01T08:00:00Z,100.00,FutBund001,ClientInternal,EUR,100,Day 1 opening price
1,2021-09-01T12:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 intraday MTM price
2,2021-09-01T17:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 closing price
3,2021-09-02T08:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 2 opening price
4,2021-09-02T14:15:00Z,99.50,FutBund001,ClientInternal,EUR,100,Day 2 intraday trade price
5,2021-09-02T17:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day 2 closing price
6,2021-09-03T08:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day 3 opening price
7,2021-09-03T14:15:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day 3 intraday trade price
8,2021-09-03T17:00:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day 3 closing price
9,2021-09-01T08:00:00Z,100.00,FutBund002,ClientInternal,EUR,100,Day 1 opening price


In [13]:
for index, row in futures_prices.iterrows():

    instrument_quotes = {
        "upsert_request_1": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["luid"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["date"],
            ),
            metric_value=models.MetricValue(value=row["prices"], unit=row["currency"]),
            scale_factor=row["scaling_factor"]            
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=quotes_scope, request_body=instrument_quotes
    )

# 5. Valuations<a class="anchor" id="-Valuations"></a>

## 5.1 Create valuation recipe

In [14]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="futuresValuation",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="Future",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

The get_daily_fut_val() function when given a date & time along with a portfolio returns a valuation of the held instruments with an indicator of their price relative to the agreed future price. This valuation therefore enables you to know if your futures are worth more or less than you agreed to pay for them at any given time.

In [15]:
def get_daily_fut_val(date, portfolio_code):
    
    metricsList=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            models.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Holding/default/Units", "Value"),
            models.AggregateSpec("Holding/default/Cost", "Value"),
            models.AggregateSpec("Valuation/PV/Amount", "Value"),
            models.AggregateSpec("Valuation/Exposure/Amount", "Value"),
        ]
        
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code="futuresValuation"),
        metrics=metricsList,
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)
    
    columnsToRename={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Instrument/Definition/ContractSize": "Contract Size",
            "Quotes/Price": "Price",
            "Holding/default/Units": "Units",
            "Holding/default/Cost": "Cost",
            "Valuation/PV/Amount": "PV",
            "Valuation/Exposure/Amount": "Exposure",
    }      

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    return vals_df

## 5.3 Daily valuations

### Day 1

#### Start of Day
At 8:00am on Day 1, we enter into our two Bund Futures contracts with starting prices of 100.00.

In [16]:
futures_transactions.iloc[0:2]

,txn_id,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
0,futa_txn_001,OpenContract,2021-09-01T08:00:00Z,10,FutBund001,EUR,100.00,1000000,FuturesPortWithDiffCostBasis
1,futb_txn_001,OpenContract,2021-09-01T08:00:00Z,10,FutBund002,EUR,100.00,1000000,FuturesPortWithDiffCostBasis


We then immediately run a start of day valuation. Our positions both have a PV of 0 and an exposure of 1,000,000 EUR

In [17]:
get_daily_fut_val("2021-09-01T08:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,0.00,"1,000,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.00,10.00,"1,000,000.00"
1,0.00,"1,000,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.00,10.00,"1,000,000.00"


#### Intraday

At 12:00pm noon, we decide to run an intraday mark to market valuation. We can see that the contract price for both instruments has moved from the original purchase price of 100.00, to 100.50. This gives us an intraday PV for both contracts of 5,000 EUR.

In [18]:
get_daily_fut_val("2021-09-01T12:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"5,000.00","1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,"1,000,000.00"
1,"5,000.00","1,005,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,10.00,"1,000,000.00"


#### End of Day

At the end of the day, we see the contract prices remain at 100.50 yielding us the same valuation we produced intraday.

In [19]:
get_daily_fut_val("2021-09-01T17:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"5,000.00","1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,"1,000,000.00"
1,"5,000.00","1,005,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,10.00,"1,000,000.00"


### Day 2
#### Start of Day

At 8:00am on Day 2, we book a 'RealisePnlIncrease' transaction in order to adjust the cost basis of the Mar 22 Bund Future upwards such that its start of day PV is 0. This adjustment can be automatically applied in LUSID removing the need to manually book these transactions each day.


In [20]:
futures_transactions.iloc[2:3]

,txn_id,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
2,rpnlb_txn_001,RealisePnLIncrease,2021-09-02T08:00:00Z,0,FutBund002,EUR,0.00,5000,FuturesPortWithDiffCostBasis


We again run a start of day valuation immediately afterwards and can see the Dec 21 Futures contract still has a PV of 5,000 EUR while the Mar 22 contract has a PV of 0. This is expected as we've now booked a cost basis adjustment upward in the amount of 5,000 EUR for the Mar 22 contract. 

In [21]:
get_daily_fut_val("2021-09-02T08:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"5,000.00","1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,"1,000,000.00"
1,0.00,"1,005,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,10.00,"1,005,000.00"
2,"5,000.00","5,000.00",EUR,None,1.00,NaN,"5,000.00","5,000.00"


#### Intraday

At 2:15pm, we decide to book a new transaction in the amount of 5 contracts of each Future at a price of 99.50. 

In [22]:
futures_transactions.iloc[3:5]

,txn_id,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
3,futa_txn_002,OpenContract,2021-09-02T14:15:00Z,5,FutBund001,EUR,99.50,497500,FuturesPortWithDiffCostBasis
4,futb_txn_002,OpenContract,2021-09-02T14:15:00Z,5,FutBund002,EUR,99.50,497500,FuturesPortWithDiffCostBasis


We then run a valuation and see that the intraday PV of the Dec 22 contract is now -5,000 EUR while the PV of the March 22 contract is now -10,000 EUR. 

##### Dec 21 (PV) = (99.50 - 100.00) \* 10 \* 100,000 / 100 + (99.50 - 99.50) \* 5 \* 100,000 / 100 =  -5,000 EUR  
Where 100.00 is the transaction price of the first transaction and where 99.50 is the transaction price of the latest transaction.

##### Mar 22 (PV) = (99.50 - 100.50) \* 10 \* 100,000 / 100 + (99.50 - 99.50) \* 5 \* 100,000 / 100 = -10,000 EUR
Where 100.50 is the EOD price from the previous day and hence the cost basis of the first transaction. 99.50 is the transaction price of the latest transaction.
<br/>

In [23]:
get_daily_fut_val("2021-09-02T14:15:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"-5,000.00","1,492,500.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",99.50,15.00,"1,497,500.00"
1,"-10,000.00","1,492,500.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",99.50,15.00,"1,502,500.00"
2,"5,000.00","5,000.00",EUR,None,1.00,NaN,"5,000.00","5,000.00"


#### End of Day

At the close of the day, both Futures prices have moved down further to 98.00 Our PVs are now computed as follows:

##### Dec 21 (PV) = (98.00 - 100.00) \* 10 \* 100,000 / 100 + (98.00 - 99.50) \* 5 \* 100,000 / 100  = -27,500 EUR
Where 100.00 is the cost basis of the first transaction and 99.50 is the cost basis of the second transaction
<br/>
##### Mar 22 (PV) = (98.00 - 100.50) \* 10 \* 100,000 / 100 + (98.00 -  99.50) \*  5 \* 100,000 / 100 = -32,500 EUR 
Where 100.50 and 99.50 are the current cost basis of the two transactions respectively.

In [24]:
get_daily_fut_val("2021-09-02T17:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"-27,500.00","1,470,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",98.00,15.00,"1,497,500.00"
1,"-32,500.00","1,470,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",98.00,15.00,"1,502,500.00"
2,"5,000.00","5,000.00",EUR,None,1.00,NaN,"5,000.00","5,000.00"


### Day 3
#### Start of Day

At 8:00am on Day 3, we book a 'RealisePnlDecrease' transaction to adjust the cost basis of the Mar 22 Bund Future downwards in the amount of yesterday's P&L giving us a start of day PV of 0.


In [25]:
futures_transactions.iloc[5:6]

,txn_id,txn_type,trade_date,quantity,client_id,currency_id,price,total_consideration,portfolio
5,rpnbl_txn_002,RealisePnLDecrease,2021-09-03T08:00:00Z,0,FutBund002,EUR,0.00,32500,FuturesPortWithDiffCostBasis


We then take a start of day valuation of the two contracts and can see the Dec 21 contract has a PV of -27,500 EUR (using the same price as previous days close) while the Mar 22 contract's PV has reset again to 0 EUR. We've updated today's cash by subtracting yesterday's PV of -32,500 EUR from yesterdays cash balance of 5,000 EUR to arrive at today's new cash balance of -27,500 EUR.
<br/><br/>
At the start of the day, we take a valuation with a price of 98.00

In [26]:
get_daily_fut_val("2021-09-03T08:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"-27,500.00","1,470,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",98.00,15.00,"1,497,500.00"
1,0.00,"1,470,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",98.00,15.00,"1,470,000.00"
2,"-27,500.00","-27,500.00",EUR,None,1.00,NaN,"-27,500.00","-27,500.00"


#### Intraday

Intraday, we take a valuation of our portfolio where the price has moved to 104.00

##### Dec 21 (PV) = (104.00 - 100.00) \* 10 \* 10,000 + (104.00 - 99.50) \* 5 \* 10,000 = 62,500 EUR  
Where 100.00 is the cost basis of the first transaction and 99.50 is the cost basis of the second transaction

##### Mar 22 (PV) = (104.00 - 98.00) \* 10 \* 10,000 +  (104.00 -  98.00) \*  5 \* 10,000 = 90,000 EUR
Where 98.0 is the current cost basis of the two corresponding transactions. 

In [27]:
get_daily_fut_val("2021-09-03T14:15:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"62,500.00","1,560,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",104.00,15.00,"1,497,500.00"
1,"90,000.00","1,560,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",104.00,15.00,"1,470,000.00"
2,"-27,500.00","-27,500.00",EUR,None,1.00,NaN,"-27,500.00","-27,500.00"


#### End of Day

Similar to our intraday valuation, we see the price hasn't changed at 104.00 and take a final end of day valuation

##### Dec 21 (PV) = (104.00 - 100.00) \* 10 \* 10,000 + (104.00 - 99.50) \* 5 \* 10,000 = 62,500 EUR  
Where 100.00 is the cost basis of the first transaction and 99.50 is the cost basis of the second transaction

##### Mar 22 (PV) = (104.00 - 98.00) \* 10 \* 10,000 +  (104.00 -  98.00) \*  5 \* 10,000 = 90,000 EUR
Where 98.0 is the current cost basis of the two corresponding transactions. 

In [28]:
get_daily_fut_val("2021-09-03T17:00:00Z", portfolio_code)

,PV,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost
0,"62,500.00","1,560,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",104.00,15.00,"1,497,500.00"
1,"90,000.00","1,560,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",104.00,15.00,"1,470,000.00"
2,"-27,500.00","-27,500.00",EUR,None,1.00,NaN,"-27,500.00","-27,500.00"
